# Prediction: Regression

In [1]:
## note: see grid_search.py for grid search code. This notebook is 
## primarily for exploration and one-off runs

## imports
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from globes import taxi_dir, days_dir
from multiprocessing import Pool, Process, cpu_count



In [2]:
FEATURE_COLS = ['pickup_day', 
                'pickup_hour', 
                'pickup_latitude', 
                'pickup_longitude']

Y_COLS = ['dropoff_latitude', 
          'dropoff_longitude']

""" get feature and label rows from filename
"""
def getXyAll(filename):
    df = pd.read_csv(filename, parse_dates=['pickup_datetime', 'dropoff_datetime'])
    
    # get rid of zones that don't actually really exist THIS IS IMPORTANT
    df = df.loc[df['pickup_zone_taxi'] != -1]
    df = df.loc[df['dropoff_zone_taxi'] != -1]
    df = df.dropna()
    
    df["pickup_day"] = df['pickup_datetime'].apply(lambda t: t.weekday())
    df["pickup_hour"] = df['pickup_datetime'].apply(lambda t: t.hour)

    df_X = df[FEATURE_COLS]
    df_Y = df[Y_COLS]
    return df_X, df_Y

""" get feature and label rows from filename, only for Manhattan
"""
def getXyManhattan(filename):
    df = pd.read_csv(filename, parse_dates=['pickup_datetime', 'dropoff_datetime'])

    # get rid of zones that don't actually really exist THIS IS IMPORTANT
    df = df.loc[df['pickup_zone_taxi'] != -1]
    df = df.loc[df['dropoff_zone_taxi'] != -1]
    df = df.dropna()
    
    # keep only Manhattan pickups
    df = df.loc[df['pickup_borough'] == "Manhattan"]
    
    df["pickup_day"] = df['pickup_datetime'].apply(lambda t: t.weekday())
    df["pickup_hour"] = df['pickup_datetime'].apply(lambda t: t.hour)

    df_X = df[FEATURE_COLS]
    df_Y = df[Y_COLS]
    return df_X, df_Y   

""" get X, y in numpy arrays from relevant data
"""
def getDataNumpy(get_Xy_func):
    num_cores = cpu_count()/2
    print "using " + str(num_cores) + " cores"
    pool = Pool(processes=num_cores)

    filenames = [os.path.join(taxi_dir, days_dir, f) for f in os.listdir(os.path.join(taxi_dir, days_dir)) if f.endswith('csv')]
    
    # get X, y dataframes in parallel
    Xy_arr = pool.map(get_Xy_func, filenames)
    pool.terminate()

    # separate out to array of X dataframes and y dataframes
    dfs_X = map(lambda pair: pair[0], Xy_arr)
    dfs_y = map(lambda pair: pair[1], Xy_arr)

    # concatenate dataframe array into single df
    df_X = pd.concat(dfs_X)
    df_y = pd.concat(dfs_y)

    # convert df to numpy arrays
    X = df_X.as_matrix()
    y = df_y.as_matrix()

    print df_X.shape, df_y.shape
    print X.shape, y.shape

    return X, y


In [3]:
X, y = getDataNumpy(getXyAll)

using 28 cores
(77699460, 4) (77699460, 2)
(77699460, 4) (77699460, 2)


In [4]:
Xm, ym = getDataNumpy(getXyManhattan)

using 28 cores
(65147846, 4) (65147846, 2)
(65147846, 4) (65147846, 2)


## Random forest regressor

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error,make_scorer
from operator import add

"""RMSE
"""
def RMSE(y, y_pred):
    rmsevalid = np.sqrt(mean_squared_error(y,y_pred, multioutput="raw_values"))
    print "RMSE: " + str(rmsevalid)
    return rmsevalid

""" Random Forest Regressor
"""
def RandomForest(X, y, split_index=69000000):
    X_train = X[:split_index]
    y_train = y[:split_index]
    
    X_test = X[split_index:]
    y_test = y[split_index:]
    
    print X_train.shape, y_train.shape, X_test.shape, y_test.shape
    
    ## best values for RMSE
    reg = RandomForestRegressor(n_estimators=8, max_depth=20, n_jobs=-1, verbose=3, warm_start=True)
        
    reg.fit(X_train, y_train)
    training_accuracy = reg.score(X_train, y_train)
    valid_accuracy = reg.score(X_test, y_test)
    rmsetrain = np.sqrt(mean_squared_error(reg.predict(X_train),y_train))
    rmsevalid = np.sqrt(mean_squared_error(reg.predict(X_test),y_test))
    
    print " R^2 (train) = %0.3f, R^2 (valid) = %0.3f, RMSE (train) = %0.3f, RMSE (valid) = %0.3f" \
        % (training_accuracy, valid_accuracy, rmsetrain, rmsevalid)
    

In [5]:
RandomForest(X,y)

(69000000, 4) (69000000, 2) (8699460, 4) (8699460, 2)
building tree 1 of 8building tree 2 of 8
building tree 5 of 8 
  
building tree 4 of 8building tree 7 of 8building tree 3 of 8building tree 6 of 8building tree 8 of 8






[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  5.3min remaining: 16.0min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  5.4min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  5.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  5.5min finished
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   18.9s remaining:   56.8s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   19.1s remaining:   11.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   19.4s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   19.4s finished
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    2.5s remaining:    7.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.6s remaining:    1.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.8s finished
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   18.7s remai

 R^2 (train) = 0.413, R^2 (valid) = 0.387, RMSE (train) = 0.029, RMSE (valid) = 0.029


In [7]:
RandomForest(Xm, ym, split_index=58000000)

(58000000, 4) (58000000, 2) (7147846, 4) (7147846, 2)
building tree 1 of 2building tree 2 of 2



[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.7min finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   17.8s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   17.8s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.9s finished
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.5s finished


 R^2 (train) = 0.257, R^2 (valid) = 0.200, RMSE (train) = 0.027, RMSE (valid) = 0.028


## SGD

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

def SGD(X, y):
    kf = KFold(n_splits=6, shuffle=True)
    res = {
        "train_r2": 0.0,
        "valid_r2": 0.0,
        "train_rmse": [0.0, 0.0],
        "valid_rmse": [0.0, 0.0]
    }
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index] 

        # scale data
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)  # apply same transformation to test data

        reg = MultiOutputRegressor(linear_model.SGDRegressor(alpha=0.0001, eta0=0.001))
        reg.fit(X_train, y_train)
        res["train_r2"] += reg.score(X_train, y_train)
        res["valid_r2"] += reg.score(X_test, y_test)
        rmse_train = RMSE(reg.predict(X_train),y_train)
        rmse_valid = RMSE(reg.predict(X_test),y_test)
        print rmse_train, rmse_valid
        res["train_rmse"] = map(add,res["train_rmse"], rmse_train)
        res["valid_rmse"] = map(add,res["valid_rmse"], rmse_valid)

    # average of error scores
    # average of error scores
    res["train_r2"] = res["train_r2"] / kf.get_n_splits(X) 
    res["valid_r2"] = res["valid_r2"] / kf.get_n_splits(X) 
    res["train_rmse"][0] = res["train_rmse"][0] / kf.get_n_splits(X) 
    res["train_rmse"][1] = res["train_rmse"][1] / kf.get_n_splits(X) 
    res["valid_rmse"][0] = res["valid_rmse"][0] / kf.get_n_splits(X) 
    res["valid_rmse"][1] = res["valid_rmse"][1] / kf.get_n_splits(X) 

    print " R^2 (train) = %0.3f, R^2 (valid) = %0.3f, RMSE (train) = %f, %f, RMSE (valid) = %f, %f" \
        % (res["train_r2"], res["valid_r2"], res["train_rmse"][0],res["train_rmse"][1], res["valid_rmse"][0],res["valid_rmse"][1])
    print ""
    

In [ ]:
SGD(X, y)

In [ ]:
SGD(Xm, ym)